In [1]:
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
import googleapiclient.errors
import pandas as pd
from IPython.display import JSON

In [21]:

api_service_name = "youtube"
api_version = "v3"
#JSON Secret file
client_secrets_file = "client_secret_ak12.json"

#Scopes
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]
# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = build(
    api_service_name, api_version, credentials=credentials)



   

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1033575135191-4kl4vd2vhahmoaod5rmbmnn9p1m1jcgk.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=MiHpzsmE6Nn5svscdJGLNg76803SYg&prompt=consent&access_type=offline
Enter the authorization code: 4/1AdQt8qiT4ZA7LC0zmH2bvjvbbykyvWohwgR_6aM5m_8UrojVOMsgffdvDBY


In [3]:
#Initial set up for first run 
ChannelCategory = "news"
#token_number = "CDIQAA"
# create empty csv
import pandas as pd
from csv import writer


def append_list_as_row(file_name, list_of_elem):
    
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [5]:
#Fetching Channel IDs-- Approach 1

def get_channel_data(youtube,ChannelCategory):
    page_iteration = 0
    all_data = []
    request = youtube.search().list(
                part="snippet",
                channelType="any",
                maxResults=50,
                q= ChannelCategory,
                order="date",
                type="channel"
                )
    response = request.execute()
    #CSV creation
    append_list_as_row('channel_data1.csv', ['channelId', 'description', 'channelTitle'])
    
    #loop through 50 items -- Following in the list
    for item in response['items']:
        data = [
            item['snippet']['channelId'],
            item['snippet']['description'],
            item['snippet']['title']
            ]
        
        #all_data.append(data) adding in csv
        append_list_as_row('channel_data1.csv', data)
    next_page_token = response.get('nextPageToken')
    print('After first 50 - next token is ', next_page_token)
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.search().list(
                        part="snippet",
                        channelType="any",
                        maxResults=50,
                        q= ChannelCategory,
                        order="date",
                        pageToken = next_page_token,
                        type="channel"
                        )
            response = request.execute()
            
            for item in response['items']:
                data = [
                    item['snippet']['channelId'],
                    item['snippet']['description'],
                    item['snippet']['title']
                    ]

                #all_data.append(data) adding in csv
                append_list_as_row('channel_data1.csv', data)
                
            next_page_token = response.get('nextPageToken')
            page_iteration = page_iteration +1
            print(next_page_token)
            if page_iteration == 20:
                more_pages = False
#                 print(next_page_token)

    
# #     while next_page_token is not None:    
    
#     while page_iteration < 50:
#         page_iteration = page_iteration + 1
#         request = youtube.search().list(
#                     part="snippet",
#                     channelType="any",
#                     maxResults=50,
#                     q= ChannelCategory,
#                     order="date",
#                     pageToken=next_page_token,
#                     type="channel"
#                     )
#         response = request.execute()

#         #loop through 50 items
#         for item in response['items']:
#             data = [
#                 item['snippet']['channelId'],
#                 item['snippet']['description'],
#                 item['snippet']['title']
#             ]
#             #all_data.append(data)

#             append_list_as_row('channel_data1.csv', data)
        
#         next_page_token = response.get('nextPageToken')
# #         if page_iteration == 80:
#         print(next_page_token)
    
        

            
    
    print(next_page_token)    
    return(pd.DataFrame(all_data))

In [6]:
channel_data = get_channel_data(youtube,ChannelCategory)

After first 50 - next token is  CDIQAA
CGQQAA
CJYBEAA
CMgBEAA
CPoBEAA
CKwCEAA
CN4CEAA
CJADEAA
CMIDEAA
CPQDEAA
None
None


In [7]:
channel_df = pd.read_csv ('channel_data1.csv', header=0 )
channel_df

,channelId,description,channelTitle
0,UCCnhx1Vqxs1K8Dfg0ZZgC_A,В современной русской мифологии NEWS 1 — бог н...,NEWS 1l
1,UCu8GW2WcGp_9Lic2wZY_IVg,熊本県内の最新ニュースのほか、気になる話題を深堀りした特集も投稿します！地上波では平日18時...,熊本朝日放送 KAB NEWS
2,UC-2L3iPWxVyvvnPLplpdCHA,愛媛県の放送局 南海放送（日本テレビ系列）の報道部が運営する公式チャンネルです。 最新ニュー...,【公式】南海放送NEWS（チャン４）
3,UCfWo0Z5PKYd86JfQ5ZE5cWg,Zee Media Corporation Limited (formerly Zee Ne...,Zee Telugu News
4,UCyFoYqSvyjVC6UC8OeTL3LA,A Rádio Sucesso News é uma empresa pertencente...,Sucesso News BSB
...,...,...,...
538,UCG9aFJTZ-lMCHAiO1KJsirg,「공정 신뢰 품격」이 있는 MBN 뉴스 ! 유튜브 MBN NEWS로 여러분께 한발 ...,MBN News
539,UC6tUZA7GiIZPaUu4FQXnFYA,Cord Cutters News is the go-to source for all ...,Cord Cutters News
540,UC86dbj-lbDks_hZ5gRKL49Q,AFP is a leading global news agency providing ...,AFP News Agency
541,UC7E-LYc1wivk33iyt5bR5zQ,Welcome to our Vatican News Channel. Get your ...,Vatican News


In [8]:
#channel_data = get_channel_data(youtube,ChannelCategory)
#channel_df = pd.read_csv('./channel_data.csv')
channel_ids = channel_df["channelId"]
channel_test = channel_ids[:50]
#channel_test = channel_ids
channel_test = channel_test.tolist()
channel_test


['UCCnhx1Vqxs1K8Dfg0ZZgC_A',
 'UCu8GW2WcGp_9Lic2wZY_IVg',
 'UC-2L3iPWxVyvvnPLplpdCHA',
 'UCfWo0Z5PKYd86JfQ5ZE5cWg',
 'UCyFoYqSvyjVC6UC8OeTL3LA',
 'UC44aP278sBev5yaGZyuUP3g',
 'UCIoTs2hgtcsoRIaPA7b-CIQ',
 'UCiMnV7jh3YGkWE73Pyb_7Hg',
 'UCHsCqnOyeTS_grW7-b5LRRw',
 'UCxiRdfyH0FtFCRZTRfRsdsA',
 'UCC-RqI6Zjh_pe_ZOSLZ-NsA',
 'UCeF5sxjXSdWq80n3RA9gBpw',
 'UCw1DLYWmcwdUsmLV9niGCdA',
 'UCrv6l7e17uoum5paG3gp4SA',
 'UCimhKtW_I9hBwtv1128PYRA',
 'UCQPjDDTlUuDfMUQoEjYmKWw',
 'UC246fseXJSAi-Q6eNtv0gbQ',
 'UCuKIRtHEg75jZRDjBbLEr0Q',
 'UCWhArxUS5mVHpIuwO7VvkLQ',
 'UC3GqXsYKXcrDiiwqoAKA-Yw',
 'UC1-aSVithJg-wd069xgXztg',
 'UCBxvxijVSfgOgCtom_yIzBw',
 'UCEQ1--skXjJNFe6px_kqjaA',
 'UCq7Ud1NU_kTNXG6ZvJrO3Jw',
 'UCAvKboOLQBzZm6K0Y3e6Wmw',
 'UCAy6QBvYZtylh7fFww-wcYw',
 'UCShjqUfgF9QuNprn2c7ceYw',
 'UCp612nMAHO497k1WNC3EfHg',
 'UC6d2gqETo6r5UfmA-0qsMUw',
 'UCbspSr6vcvSitU407uvKNSw',
 'UCLZlV0OGIWXJRZQl25iVcXQ',
 'UCVsxK4nyxgbF0aXsoPd6pDA',
 'UCiKkWd87ABufhjjKwBY3cNQ',
 'UCH_zBfX_o5GDZwevkh1jl9Q',
 'UCoE9pRJB9Tt

In [9]:
request = youtube.channels().list(
    part="statistics, contentDetails",
    id=','.join(channel_test),
    maxResults=50
    )
response = request.execute()

print(response)

{'kind': 'youtube#channelListResponse', 'etag': '6bWWAcsw-8chMN5TWy4xa6T808w', 'pageInfo': {'totalResults': 50, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#channel', 'etag': 'cRd9cHTRoJmv0gUzwMOCPm7e7Nc', 'id': 'UCC-RqI6Zjh_pe_ZOSLZ-NsA', 'contentDetails': {'relatedPlaylists': {'likes': '', 'uploads': 'UUC-RqI6Zjh_pe_ZOSLZ-NsA'}}, 'statistics': {'viewCount': '5528', 'hiddenSubscriberCount': True, 'videoCount': '4'}}, {'kind': 'youtube#channel', 'etag': 'k0aqfdtdVuVFbV2uhDwRhp5vMWQ', 'id': 'UCk8QYKyLJoOkgshAVUwIP_w', 'contentDetails': {'relatedPlaylists': {'likes': '', 'uploads': 'UUk8QYKyLJoOkgshAVUwIP_w'}}, 'statistics': {'viewCount': '16332844', 'hiddenSubscriberCount': True, 'videoCount': '892'}}, {'kind': 'youtube#channel', 'etag': 'L7BLAnkG-nIyqEjE1dT1419cvfE', 'id': 'UCxiRdfyH0FtFCRZTRfRsdsA', 'contentDetails': {'relatedPlaylists': {'likes': '', 'uploads': 'UUxiRdfyH0FtFCRZTRfRsdsA'}}, 'statistics': {'viewCount': '12800509', 'subscriberCount': '15400', 'hiddenSubscriberCou

In [10]:
#fetching channels subscribers

def get_channel_sub_data(youtube,channel_test):
    all_data = []
    request = youtube.channels().list(
        part="statistics, contentDetails",
        id=','.join(channel_test),
        maxResults=50
        )
        
        
    response = request.execute()
    append_list_as_row('channel_sub_data12.csv', ['channelId', 'SubscriberCount', 'ViewCount', 'hiddenSubscriberCount', 'videoCount'])
    #
    #loop through items
    #Following in the list
    for item in response['items']:
        
        if item['statistics']['hiddenSubscriberCount'] == False :
            data = [
                item['id'],
                item['statistics']['subscriberCount'] ,
                item['statistics']['viewCount'],
                item['statistics']['hiddenSubscriberCount'],
                item['statistics']['videoCount']
                ]
        
        
            append_list_as_row('channel_sub_data12.csv', data)
    

    return(pd.DataFrame(all_data))




In [11]:

channel_sub = get_channel_sub_data(youtube,channel_test)
channel_sub_df = pd.read_csv ('channel_sub_data12.csv')
channel_sub_df


,channelId,SubscriberCount,ViewCount,hiddenSubscriberCount,videoCount
0,UCq7Ud1NU_kTNXG6ZvJrO3Jw,189000,85619301,False,15846
1,UCp612nMAHO497k1WNC3EfHg,60600,6035127,False,68
2,UCrv6l7e17uoum5paG3gp4SA,1360,469635,False,429
3,UCLZlV0OGIWXJRZQl25iVcXQ,53300,6559494,False,187
4,UCoE9pRJB9TtlEm1RzY9vPdA,32300,3537543,False,4359
5,UCBxvxijVSfgOgCtom_yIzBw,27,17777,False,587
6,UCIoTs2hgtcsoRIaPA7b-CIQ,14900,4177809,False,89
7,UCQPjDDTlUuDfMUQoEjYmKWw,20000,1229592,False,145
8,UCShjqUfgF9QuNprn2c7ceYw,20300,1646951,False,65
9,UC3GqXsYKXcrDiiwqoAKA-Yw,120000,13499245,False,77


In [12]:
#Fetching videosID from channelID
Channel_list = "UCeY0bbntWzzVIaj2z3QigXg"

def get_video_data(youtube,Channel_list):
    all_data = []
    request = youtube.search().list(
        part="snippet",
        channelId= Channel_list,
        maxResults=25,
        type="video",
        videoType="any"
    )
    response = request.execute()
    append_list_as_row('video_data.csv', ['channelId', 'videoId', 'description', 'videoTitle' ])

    #loop through items
    for item in response['items']:
        data = [
            item['snippet']['channelId'],
            item['id']['videoId'],
            item['snippet']['description'],
            item['snippet']['title']
        ]
        #all_data.append(data)
        append_list_as_row('video_data.csv', data)
        
    
    
    
    
    return(pd.DataFrame(all_data))

In [26]:
video_data = get_video_data(youtube,Channel_list)
video_df = pd.read_csv('./video_data.csv')
video_ids = video_df["videoId"].tolist()
#video_data

In [27]:
video_df

,channelId,videoId,description,videoTitle
0,UCeY0bbntWzzVIaj2z3QigXg,ZVK3nEoJSJ8,Ukraine takes back a village outside Kharkiv f...,Nightly News Full Broadcast - March 30
1,UCeY0bbntWzzVIaj2z3QigXg,Qjxs4x3_-Fw,18:48 U.S. says Ukraine missiles sank a Russia...,Nightly News Full Broadcast - April 15
2,UCeY0bbntWzzVIaj2z3QigXg,VnKIzr4eFUM,"NBC News NOW is live, reporting breaking news ...",LIVE: NBC News NOW
3,UCeY0bbntWzzVIaj2z3QigXg,fK91IORzxeU,"Pfizer testing omicron-specific Covid vaccine,...",Nightly News Full Broadcast - Jan. 25
4,UCeY0bbntWzzVIaj2z3QigXg,KDvZR_I8ehw,As President Donald Trump and Kim Jong Un sit ...,Inside The Unlikely Friendship Of Kim Jong Un ...
...,...,...,...,...
98,UCeY0bbntWzzVIaj2z3QigXg,UDmPBFUH_Uc,Missile strike on train station in eastern Ukr...,Nightly News Full Broadcast - April 8
99,UCeY0bbntWzzVIaj2z3QigXg,YYxJ8UcoBDY,"Russia captures Kherson, Lester Holt's exclusi...",Nightly News Full Broadcast - March 3
100,UCeY0bbntWzzVIaj2z3QigXg,xHQTDEN7Xek,Molotov cocktail thrown at Wisconsin anti-abor...,Nightly News Full Broadcast - May 8
101,UCeY0bbntWzzVIaj2z3QigXg,bPSGOLUynRc,On tonight's broadcast: The travel industry is...,Nightly News Full Broadcast - July 1


In [28]:
#Fetching video details like viewCount ; likeCount; commentCount
#video_ids = "ZVK3nEoJSJ8"
def get_video_info(youtube,video_ids):
    all_data = []
    request = youtube.videos().list(
        part="statistics",
        id=','.join(video_ids)
        )
    response = request.execute()

    #loop through items
    for item in response['items']:
        data = {
            'VideoId': item['id'],
            'ViewCount': item['statistics']['viewCount'],
            'LikeCount': item['statistics']['likeCount'],
            'CommentCount': item['statistics']['commentCount']
            }
        #all_data.append(data)
        append_list_as_row('video_info.csv', data)
    return(pd.DataFrame(all_data))


In [24]:
# video_ids = video_df["videoId"].tolist()
video_info = get_video_info(youtube,video_ids)
video_info

['ZVK3nEoJSJ8',
 'Qjxs4x3_-Fw',
 'VnKIzr4eFUM',
 'fK91IORzxeU',
 'KDvZR_I8ehw',
 'mFgfTVkFV9I',
 '1BgajNI3UTs',
 'AK5EBfj33ls',
 'izMhP-YhN0g',
 'FYHQlUaWIhE',
 'JgsG7KvJUT4',
 'ATBVL2ikiV4',
 'Rv6LzZoLlm0',
 'Dm2FQviiuS4',
 'qKpKgnuT3ZI',
 'CjhynYLD4-k',
 '_2wjPbdut9E',
 'aiqGBQDz4eQ',
 '97HECjz8aUA',
 'P1iE5SfkhBs',
 'SVB8_o7MKyQ',
 'L89patMuCPE',
 '8EMormCU0HA',
 'GXuqBJ8W-PI',
 'UDmPBFUH_Uc',
 'videoId',
 'ZVK3nEoJSJ8',
 'Qjxs4x3_-Fw',
 'Cia1dFW3kFs',
 'fK91IORzxeU',
 'mFgfTVkFV9I',
 'AK5EBfj33ls',
 'FYHQlUaWIhE',
 'JgsG7KvJUT4',
 'WohvKxCDfHs',
 'izMhP-YhN0g',
 'Dm2FQviiuS4',
 'vw9TpckvZF0',
 'lG_SR-liDE4',
 'P1iE5SfkhBs',
 'hw2TmDCr7ic',
 'YysDh-qvu0A',
 '0RX2mLv1UJc',
 'ZA-h3dIeD_A',
 'L89patMuCPE',
 'ygKeLBif-7I',
 'UDmPBFUH_Uc',
 'YYxJ8UcoBDY',
 'xHQTDEN7Xek',
 'bPSGOLUynRc',
 'yTjsoK5sIek']

In [ ]:
def get_video_data(youtube,long_lat):
    all_data = []
    request = youtube.search().list(
        part="snippet",
        location= long_lat,
        locationRadius="10mi",
        type="video",
        videoCategoryId="25"
        )
    response = request.execute()
    
    #loop through items
    for item in response['items']:
        data = {
            'channelId': item['snippet']['channelId'],
            'description': item['snippet']['description'],
            'channelTitle': item['snippet']['title']
            }
    all_data.append(data)
    return(pd.DataFrame(all_data))

In [ ]:

video_data = get_video_data(youtube,long)
video_data

In [ ]:
video_data = get_video_data(youtube,long_lat)
video_data

In [ ]:


def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
        )
    response = request.execute()
    
    #loop through items
    for item in response['items']:
        data = {
            'VideoTitle': item['snippet']['title'],
            'subscribers': item['snippet']['subscriberCount'],
            'ChannelTitle': item['snippet']['title']
            
            }
    all_data.append(data)
    return(pd.DataFrame(all_data))

In [ ]:
#Fetching Channel IDs-- Approach 2
#Fetching Channel - Approach 2

def get_channel_data(youtube,ChannelCategory):
    page_iteration = 0
    all_data = []
    request = youtube.search().list(
                part="snippet",
                channelType="any",
                maxResults=50,
                q= ChannelCategory,
                order="date",
                type="channel"
                )
    response = request.execute()
    
    #CSV creation
    append_list_as_row('channel_data1.csv', ['channelId', 'description', 'channelTitle'])
    
    
    for item in response['items']:
        data = [
            item['snippet']['channelId'],
            item['snippet']['description'],
            item['snippet']['title']
            ]
        append_list_as_row('channel_data1.csv', data)
    
    next_page_token = response.get('nextPagetoken')
    print('After first 50 - next token is ', next_page_token)
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.search().list(
                        part="snippet",
                        channelType="any",
                        maxResults=50,
                        q= ChannelCategory,
                        order="date",
                        pageToken = next_page_token,
                        type="channel"
                        )
            response = request.execute()
            
            for item in response['items']:
                data = [
                    item['snippet']['channelId'],
                    item['snippet']['description'],
                    item['snippet']['title']
                    ]

                #all_data.append(data) adding in csv
                append_list_as_row('channel_data1.csv', data)
                
            next_page_token = response.get('nextPageToken')
            page_iteration = page_iteration +1
            if page_iteration == 10:
                more_pages = False
                print('Last token is ',next_page_token)
                
                
        
        
    return(pd.DataFrame(all_data))

In [ ]:
channel_data = get_channel_data(youtube,ChannelCategory)